<a href="https://colab.research.google.com/github/niranjana2222/Pose-Classification/blob/main/HPEClassificationModel_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 
import tensorflow as tf

import os

import random

In [ ]:
# Accessing the training and testing data (NEED TO FIX THIS PART)

train_dir = '../input/yoga-poses-dataset/DATASET/TRAIN'
test_dir = '../input/yoga-poses-dataset/DATASET/TEST'


In [ ]:
# Plot a few of the training images of each class

def plot_yoga_images(train_dir):
    plt.figure(figsize = (15,15))
    for i, col in enumerate(os.listdir(train_dir)):
        image = random.choice(os.listdir(train_dir + '/' + col))
        image_path = train_dir + '/' + col + '/' + image
        img = mpimg.imread(image_path)/255

        plt.subplot(3,3,i+1)
        plt.title(col)
        plt.imshow(img)
        plt.axis(False)
        i = i + 1

plot_yoga_images(train_dir)

In [ ]:
# Preprocessing

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# creating data loaders for preparing the data 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32 

# createing data generators 
train_datagen = ImageDataGenerator(rescale = 1 / 255.)
test_datagen = ImageDataGenerator(rescale = 1 / 255.)

print('Training Images')
train_data = train_datagen.flow_from_directory(
  train_dir,
  target_size = IMAGE_SHAPE,
  batch_size = BATCH_SIZE,
  class_mode = 'categorical'
)

print('Testing Images')
test_data = test_datagen.flow_from_directory(
  test_dir,
  target_size = IMAGE_SHAPE,
  batch_size = BATCH_SIZE,
  class_mode = 'categorical'
)

In [ ]:
# import tensorflow and create the model

import tensorflow_hub as hub 
from tensorflow.keras import layers

efficientnet_url = 'https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1'
resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5'


# function for creating model
def create_model(model_url, num_classes = 2):
  
  feature_extractor_layer = hub.KerasLayer(
      model_url,
      trainable = False, 
      name = 'feature_extraction_layer',
      input_shape = IMAGE_SHAPE + (3,)
  )
  model = tf.keras.Sequential([
    feature_extractor_layer,
    layers.Dense(
        num_classes, 
        activation = 'softmax',
        name = 'output_layer'
    )
  ])

  return model 

In [ ]:
# function to plot the loss

def plot_loss(history):
      model_df = pd.DataFrame(history.history)
      loss = model_df.loss
      val_loss = model_df.val_loss
      accuracy = model_df.accuracy
      val_accuracy = model_df.val_accuracy

      epochs = range(len(model_df.loss))
      # plotting loss 
      plt.figure(figsize = (7,5))
      plt.title(
          'Loss'
      )
      plt.xlabel(
          'Epochs'
      )
      plt.plot(
          epochs,
          loss, 
          label = 'Training loss'
      )
      plt.plot(
          epochs, 
          val_loss, 
          label = 'Validation loss'
      )
      plt.legend()

      # plotting accuracy 
      plt.figure(figsize=(7,5))
      plt.title(
          'Accuracy'
      )
      plt.xlabel(
          'Epochs'
      )
      plt.plot(
          epochs, 
          accuracy,
          label = 'training accuracy',
      )
      plt.plot(
          epochs, 
          val_accuracy,
          label = 'validation accuracy'
      )
      plt.legend()

In [ ]:
# training the ResNet model

resnet_model = create_model(
    resnet_url,
    num_classes = train_data.num_classes
)

# Compile the model 
resnet_model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

# Fitting the model 
resnet_history = resnet_model.fit(
    train_data,
    epochs = 5,
    steps_per_epoch = len(train_data),
    validation_data = test_data,
    validation_steps = len(test_data)
)

# Plot the loss

plot_loss(resnet_history)
